In [1]:
import os, re, sys, math, base64
from openai import OpenAI
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
current_directory = os.getcwd()

In [3]:
# Load the image lookup CSV file
ganfd = pd.read_csv('../GANFD/image_lookup.csv')

In [4]:
os.chdir(os.path.join(current_directory, "../GANFD/Images"))
images = [f for f in sorted(os.listdir(os.getcwd())) if f.endswith('.jpg')]
image_dir = {image: os.path.join(current_directory, "../GANFD/Images", image) for image in images}
ganfd['full_ID_with_ext'] = ganfd['full_ID'] + '.jpg'
ganfd['directory'] = ganfd['full_ID_with_ext'].map(image_dir)
ganfd = ganfd.drop(columns=['full_ID_with_ext'])
ganfd_dict = ganfd.set_index('full_ID')['directory'].to_dict()

## Define Instructions and Prompts

In [5]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [6]:
prompt_dict = {
    "Story": {
        "instruction": "You are a helpful chat assistant. You are going to generate texts in response to images depicting fictional individuals.",
        "prompt": "Write a 50-word story about an individual inside the image."
    }
}

## Define Measures

In [7]:
def calculate_pd(probs): 
    prob_diff = 1 - sum(x**2 for x in probs)
    return prob_diff

def calculate_entropy(probs):
    return -np.sum(probs * np.log(probs))

def calculate_perplexity(probs):
    ent = calculate_entropy(probs)
    perplexity = 2 ** ent
    return perplexity

## Data Collection

In [8]:
client = OpenAI(
    api_key = ""
)

In [9]:
num_iter = 50
df_list = []

In [10]:
df_list = []

for image_key, image_path in ganfd_dict.items():
    base64_image = encode_image(image_path)
    
    # Create a list of messages for num_iter iterations
    messages = [
        {"role": "system", "content": prompt_dict["Story"]["instruction"]},
        {"role": "user", "content": [
            {"type": "text", "text": prompt_dict["Story"]["prompt"]},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
        ]}
    ]

    # Request with n = num_iter
    response = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",  
        messages=messages,
        n=num_iter,
        max_tokens=50,  # max_tokens should be same as token_index
        logprobs=True,
        top_logprobs=20  # max value
    )

    token_index = 50

    # Initialize lists
    pd_list = []
    entropy_list = []
    perplexity_list = []
    index_list = []
    image_list = []
    iter_list = []
    text_list = []

    # Process all iterations in the response
    for iter in range(num_iter):
        try:
            # Extract the text response for this iteration
            full_text = response.choices[iter].message.content
            
            for index in range(token_index):
                probs = [np.exp(i.logprob) for i in response.choices[iter].logprobs.content[index].top_logprobs]
                iter_list.append(iter)
                index_list.append(index)
                pd_list.append(calculate_pd(probs))
                entropy_list.append(calculate_entropy(probs))
                perplexity_list.append(calculate_perplexity(probs))
                image_list.append(image_key)
                text_list.append(full_text)  # Add the full response text for each token
        except IndexError:
            continue

    # Create DataFrame for the current image and append to df_list
    df = pd.DataFrame({
        'image': image_list,
        'iter': iter_list,
        'index': index_list,
        'pd': pd_list,
        'entropy': entropy_list,
        'perplexity': perplexity_list,
        'text': text_list  # Include the full response text column
    })
    df_list.append(df)

# Combine all DataFrames into a single DataFrame
final_df = pd.concat(df_list, ignore_index=True)

In [11]:
os.chdir(current_directory)
final_df.to_csv('gpt4turbo.csv', index = False)